# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.43MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [4]:
import pandas as pd
import os
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [5]:
df = pd.read_csv('/content/Dataset.csv')
df

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


In [6]:
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [7]:
df.tail()

,المخالفة,الغرامة
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال
103,المجازفة بعبور الأودية والشعاب أثناء جريانها.,الغرامة المالية 5000 - 10000 ريال


In [8]:
df.sample()

,المخالفة,الغرامة
67,عدم إخراج المركبة المعدة للتصدير خلال المدة ال...,الغرامة المالية 1000 - 2000 ريال


In [9]:
df.isna().sum()

,0
المخالفة,0
الغرامة,0


In [10]:
df.duplicated().sum()

0

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [12]:
df['المخالفة'].str.len().max()

141

In [13]:
df['الغرامة'].str.len().max()

33

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [14]:
dir = '/content/data/markdown_files'
os.makedirs(dir, exist_ok=True)

In [15]:
for i in range(0, len(df)):
  title = df['المخالفة'].iloc[i]
  context = df['الغرامة'].iloc[i]

  markdown_text = f'# {title}\n'
  markdown_text += f'{context}'

  with open(f'{dir}/{i}.md', 'w', encoding='utf-8') as file:
    file.write(markdown_text)

In [16]:
all_files_html = []

for filename in os.listdir(dir):
  if filename.endswith('.md'):
    with open(os.path.join(dir, filename),'r',encoding='utf-8') as file:
      read_files = file.read()
      html_content = markdown.markdown(read_files)
      all_files_html.append(html_content)

In [17]:
all_files_html

['<h1>ترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد.</h1>\n<p>الغرامة المالية 300 - 500 ريال</p>',
 '<h1>استعمال المكابح (الفرامل) بشكل مفاجئ لغير ضرورة.</h1>\n<p>الغرامة المالية 300 - 500 ريال</p>',
 '<h1>عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>',
 '<h1>قيادة المركبة بالاتجاه المعاكس لحركة السير.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 '<h1>الوقوف على خطوط السكة الحديدية.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 '<h1>نقل عدد من الركاب يزيد على العدد المحدد في رخصة السير.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 '<h1>استخدام السائق بيده أي جهاز محمول أثناء سير المركبة.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>',
 '<h1>تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 '<h1>عدم قيام أصحاب الحيوانات بإبعادها عن حرم الطريق المعتمد.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 '<h1>طمس لوحا

In [18]:
len(all_files_html)

104

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [19]:
splitter = RecursiveCharacterTextSplitter(chunk_size=60, chunk_overlap=15)
splitted_docs = splitter.create_documents(all_files_html)

In [20]:
splitted_docs

[Document(metadata={}, page_content='<h1>ترك الأطفال دون سن العاشرة في المركبة دون مرافق'),
 Document(metadata={}, page_content='دون مرافق راشد.</h1>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 300 - 500 ريال</p>'),
 Document(metadata={}, page_content='<h1>استعمال المكابح (الفرامل) بشكل مفاجئ لغير ضرورة.</h1>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 300 - 500 ريال</p>'),
 Document(metadata={}, page_content='<h1>عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها'),
 Document(metadata={}, page_content='أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود'),
 Document(metadata={}, page_content='حال عدم وجود لوحات تنظم ذلك.</h1>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 500 - 900 ريال</p>'),
 Document(metadata={}, page_content='<h1>قيادة المركبة بالاتجاه المعاكس لحركة السير.</h1>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 3000 - 6000 ريال</p>'),
 Document(metadata={}, page_content='<h1>الوقوف على خطوط السكة الحديدي

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [22]:
embedding_function = SentenceTransformerEmbeddings(model_name='Omartificial-Intelligence-Space/GATE-AraBert-v0')
data_base = Chroma.from_documents(splitted_docs, embedding_function, persist_directory='./Chroma_Data_Base')

<ipython-input-22-831a81824a3a>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name='Omartificial-Intelligence-Space/GATE-AraBert-v0')
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [46]:
prompt_template = """
##التعليمات
*أنت نظام المرور في المملكة العربية السعودية وسيتم تزويدك بالسياق
*يجب عليك عدم الخروج عن المعلومات المسجلة في النظام
*اذا كان السياق المزود لك غير ذات صلة، يمكنك اخبار المستخدم بإنك لا تعرف الاجابة
* يجب عليك ارجاع الغرامة بشكل صحيح حسب نظام المرور وحسب سؤال المستخدم
* يجب عليك تزويد الاجابة باللغة العربية

قد يسأل المستخدم عن سؤال خارج نظام المرور، لذلك يمكنك الامتناع عن الاجابة

أجب على السؤال بناء على التالي
السياق: {context}
السؤال: {question}
جوابك:
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=['context', 'question']
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [47]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [48]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [49]:
def query_rag(query: str):
    similarity_search = data_base.similarity_search_with_score(query, k=5)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [50]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")
response

Prompt after formatting:

##التعليمات
*أنت نظام المرور في المملكة العربية السعودية وسيتم تزويدك بالسياق 
*يجب عليك عدم الخروج عن المعلومات المسجلة في النظام
*اذا كان السياق المزود لك غير ذات صلة، يمكنك اخبار المستخدم بإنك لا تعرف الاجابة
* يجب عليك ارجاع الغرامة بشكل صحيح حسب نظام المرور وحسب سؤال المستخدم
* يجب عليك تزويد الاجابة باللغة العربية

قد يسأل المستخدم عن سؤال خارج نظام المرور، لذلك يمكنك الامتناع عن الاجابة

أجب على السؤال بناء على التالي
السياق: <h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع

<h1>عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.</h1>

جسم المركبة دون موافقة الجهات المختصة.</h1>

<h1>قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح

<h1>عدم وجود تأمين ساري للمركبة.</h1>
السؤال: ماهي الغرامة على القيادة بدون رخصة؟
جوابك:


> Finished chain.


{'context': '<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع\n\n<h1>عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.</h1>\n\nجسم المركبة دون موافقة الجهات المختصة.</h1>\n\n<h1>قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح\n\n<h1>عدم وجود تأمين ساري للمركبة.</h1>',
 'question': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': 'وفقاً لما هو مقرر في نظام المرور السعودي، الغرامة على القيادة بدون رخصة هي 500 ريال سعودي، بالإضافة إلى سحب رخصة القيادة لمدة سنة واحدة.'}

In [51]:
response = query_rag("ماهي الغرامة على عدم وجود تأمين ساري للمركبة؟")
response

Prompt after formatting:

##التعليمات
*أنت نظام المرور في المملكة العربية السعودية وسيتم تزويدك بالسياق 
*يجب عليك عدم الخروج عن المعلومات المسجلة في النظام
*اذا كان السياق المزود لك غير ذات صلة، يمكنك اخبار المستخدم بإنك لا تعرف الاجابة
* يجب عليك ارجاع الغرامة بشكل صحيح حسب نظام المرور وحسب سؤال المستخدم
* يجب عليك تزويد الاجابة باللغة العربية

قد يسأل المستخدم عن سؤال خارج نظام المرور، لذلك يمكنك الامتناع عن الاجابة

أجب على السؤال بناء على التالي
السياق: <h1>عدم وجود تأمين ساري للمركبة.</h1>

<h1>عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة في حالات

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع

<h1>قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح

جسم المركبة دون موافقة الجهات المختصة.</h1>
السؤال: ماهي الغرامة على عدم وجود تأمين ساري للمركبة؟
جوابك:


> Finished chain.


{'context': '<h1>عدم وجود تأمين ساري للمركبة.</h1>\n\n<h1>عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة في حالات\n\n<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع\n\n<h1>قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح\n\nجسم المركبة دون موافقة الجهات المختصة.</h1>',
 'question': 'ماهي الغرامة على عدم وجود تأمين ساري للمركبة؟',
 'text': 'وفقاً لما هو مقرر في نظام المرور السعودي، فإن الغرامة على عدم وجود تأمين ساري للمركبة هي 500 ريال سعودي.'}

In [52]:
response = query_rag("ماهي رؤية 2030؟")
response

Prompt after formatting:

##التعليمات
*أنت نظام المرور في المملكة العربية السعودية وسيتم تزويدك بالسياق 
*يجب عليك عدم الخروج عن المعلومات المسجلة في النظام
*اذا كان السياق المزود لك غير ذات صلة، يمكنك اخبار المستخدم بإنك لا تعرف الاجابة
* يجب عليك ارجاع الغرامة بشكل صحيح حسب نظام المرور وحسب سؤال المستخدم
* يجب عليك تزويد الاجابة باللغة العربية

قد يسأل المستخدم عن سؤال خارج نظام المرور، لذلك يمكنك الامتناع عن الاجابة

أجب على السؤال بناء على التالي
السياق: <h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين

القادمة من الاتجاهات الأخرى.</h1>

الرؤية غير واضحة.</h1>

التي تضعها الإدارة المختصة.</h1>

وجود توجيه أو علامة توجب الوقوف.</h1>
السؤال: ماهي رؤية 2030؟
جوابك:


> Finished chain.


{'context': '<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين\n\nالقادمة من الاتجاهات الأخرى.</h1>\n\nالرؤية غير واضحة.</h1>\n\nالتي تضعها الإدارة المختصة.</h1>\n\nوجود توجيه أو علامة توجب الوقوف.</h1>',
 'question': 'ماهي رؤية 2030؟',
 'text': 'أنا نظام المرور في المملكة العربية السعودية، ولا أملك معلوماتاً عن رؤية 2030، فلا أستطيع الإجابة على هذا السؤال لأنها غير ذات صلة بالسياق المزود لي.'}